In [1]:
#from google.colab import drive
#drive.mount('/content/drive')
import sys
#PATH = '/content/drive/My Drive/PPM_Stability/'
#PATH = "C:/Users/velmurug/Documents/Stability Experiments/benchmark_interpretability/PPM_Stability/"
#PATH = "C:/Users/Mythreyi/Documents/GitHub/Stability-Experiments/benchmark_interpretability/PPM_Stability/"
PATH = "C:/Users/mythr/Documents/GitHub/Stability-Experiments/benchmark_interpretability/PPM_Stability/"
sys.path.append(PATH)

In [2]:
#!pip install lime
#!pip install shap
#!pip install pandas==0.19.2
!pip install xgboost==1.0.0

In [3]:
import EncoderFactory
from DatasetManager import DatasetManager
import BucketFactory
import stability as st #Nogueira, Sechidis, Brown.

import pandas as pd
import numpy as np
from scipy import stats

from sklearn.metrics import roc_auc_score
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.preprocessing import StandardScaler

import time
import os
import sys
from sys import argv
import pickle
from collections import defaultdict
import random
import joblib

from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

import lime
import lime.lime_tabular
from lime import submodular_pick;

import shap

import warnings
warnings.filterwarnings('ignore')

In [4]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

def imp_df(column_names, importances):
        df = pd.DataFrame({'feature': column_names,
                       'feature_importance': importances}) \
           .sort_values('feature_importance', ascending = False) \
           .reset_index(drop = True)
        return df

# plotting a feature importance dataframe (horizontal barchart)
def var_imp_plot(imp_df, title, num_feat):
        imp_df.columns = ['feature', 'feature_importance']
        b= sns.barplot(x = 'feature_importance', y = 'feature', data = imp_df.head(num_feat), orient = 'h', palette="Blues_r")

In [5]:
def generate_global_explanations(train_X,train_Y, cls, feature_combiner):
    
    print("The number of testing instances is ",len(train_Y))
    print("The total number of columns is",train_X.shape[1]);
    print("The total accuracy is ",cls.score(train_X,train_Y));
       
    sns.set(rc={'figure.figsize':(10,10), "font.size":18,"axes.titlesize":18,"axes.labelsize":18})
    sns.set
    feat_names = feature_combiner.get_feature_names()
    base_imp = imp_df(feat_names, cls.feature_importances_)
    base_imp.head(15)
    var_imp_plot(base_imp, 'Feature importance using XGBoost', 15)
    return base_imp

In [6]:
from lime import submodular_pick
def generate_lime_explanations(explainer,test_xi, cls,test_y, submod=False, test_all_data=None, max_feat = 10):
    
    #print("Actual value ", test_y)
    exp = explainer.explain_instance(test_xi, 
                                 cls.predict_proba, num_features=max_feat, labels=[0,1])
    
    return exp
        
    if submod==True:
        sp_obj=submodular_pick.SubmodularPick(explainer, test_all_data, cls.predict_proba, 
                                      sample_size=20, num_features=num_features,num_exps_desired=4)
        [exp.as_pyplot_figure(label=exp.available_labels()[0]) for exp in sp_obj.sp_explanations];

In [7]:
def create_samples(shap_explainer, iterations, row, features, top = None):
    length = len(features)
    
    exp = []
    rel_exp = []
    
    for j in range(iterations):
        
        #if shap_type == "kernel":
        #    shap_explainer = shap.KernelExplainer(cls.predict, trainingsample)
        #elif shap_type == "tree":
        #    shap_explainer = shap.TreeExplainer(cls)
        #elif shap_type == "deep":
        #    shap_explainer = shap.DeepExplainer(cls, background)
        
        #print(X_test_frame.loc[row])
        shap_values = shap_explainer.shap_values(row)
        #print(shap_values)

        importances = []
        
        if type(shap_explainer) == shap.explainers.kernel.KernelExplainer:
            for i in range(length):
                feat = features[i]
                shap_val = shap_values[0][i]
                abs_val = abs(shap_values[0][i])
                entry = (feat, shap_val, abs_val)
                importances.append(entry)
                
        elif type(shap_explainer) == shap.explainers.tree.TreeExplainer:
            for i in range(length):
                feat = features[i]
                shap_val = shap_values[0][i]
                abs_val = abs(shap_values[0][i])
                entry = (feat, shap_val, abs_val)
                importances.append(entry)
        
        elif type(shap_explainer) == shap.explainers.deep.DeepExplainer:
            for i in range(length):
                feat = features[i]
                shap_val = shap_values[0][0][i]
                abs_val = abs(shap_values[0][0][i])
                entry = (feat, shap_val, abs_val)
                importances.append(entry)
        
        importances.sort(key=lambda tup: tup[2], reverse = True)
        
        exp.append(importances)

        rel_feat = []

        if top != None:
            for i in range(top):
                feat = importances[i]
                if feat[2] > 0:
                    rel_feat.append(feat)

            rel_exp.append(rel_feat)
        else:
            rel_exp = exp
        
    return exp, rel_exp

In [8]:
def dispersal(weights, features):
    feat_len = len(features)
    #print(feat_len)
    weights_by_feat = []
       
    for i in list(range(feat_len)):
        feat_weight = []
        for iteration in weights:
            feat_weight.append(iteration[i])
        weights_by_feat.append(feat_weight)
        
    #for iteration in weights:
     #   for val in iteration:
      #      idx = iteration.index(val)
       #     print(idx)
        #    weights_by_feat[idx].append(val)
    
    dispersal = []
    dispersal_no_outlier = []
    
    for each in weights_by_feat:
        #print("Feature", weights_by_feat.index(each)+1)
        mean = np.mean(each)
        std_dev = np.std(each)
        var = std_dev**2
        
        if mean == 0:
            dispersal.append(0)
            dispersal_no_outlier.append(0)
        #print(each)
        else:
            #dispersal with outliers
            rel_var = var/abs(mean)
            dispersal.append(rel_var)
            
            #dispersal without outliers - remove anything with a z-score higher
            #than 3 (more than 3 standard deviations away from the mean)
            rem_outlier = []
            z_scores = stats.zscore(each)
            #print(z_scores)
            #print("New list:")
            for i in range(len(z_scores)):
                #print(each[i],":",z_scores[i])
                if -3 < z_scores[i] < 3:
                    rem_outlier.append(each[i])
                #print(rem_outlier)
            if rem_outlier != []:
                new_mean = np.mean(rem_outlier)
                if new_mean == 0:
                    dispersal_no_outlier.append(0)
                else:
                    new_std = np.std(rem_outlier)
                    new_var = new_std**2
                    new_rel_var = new_var/abs(new_mean)
                    dispersal_no_outlier.append(new_rel_var)
            else:
                dispersal_no_outlier.append(rel_var)
    #print(dispersal_no_outlier)
    return dispersal, dispersal_no_outlier

In [9]:
dataset_ref = "bpic2012"
params_dir = PATH + "params"
results_dir = "results"
bucket_method = "single"
cls_encoding = "agg"
cls_method = "xgboost"

gap = 1
n_iter = 1

method_name = "%s_%s"%(bucket_method, cls_encoding)

generate_samples = False
generate_lime = True
generate_kernel_shap = False
generate_model_shap = True

sample_size = 2
exp_iter = 10
max_feat = 10

dataset_ref_to_datasets = {
    #"bpic2011": ["bpic2011_f%s"%formula for formula in range(1,5)],
    "bpic2015": ["bpic2015_%s_f2"%(municipality) for municipality in range(5,6)],
    "bpic2017" : ["bpic2017_accepted"],
    "bpic2012" : ["bpic2012_accepted"]
    #"insurance": ["insurance_activity", "insurance_followup"],
    #"sepsis_cases": ["sepsis_cases_1", "sepsis_cases_2", "sepsis_cases_4"]
}

datasets = [dataset_ref] if dataset_ref not in dataset_ref_to_datasets else dataset_ref_to_datasets[dataset_ref]

datasets

['bpic2012_accepted']

In [10]:
for dataset_name in datasets:

    dataset_manager = DatasetManager(dataset_name)

    for ii in range(n_iter):
        num_buckets = range(len([name for name in os.listdir(os.path.join(PATH,'%s/%s_%s/models'% (dataset_ref, cls_method, method_name)))]))

        for bucket in list(num_buckets):
            bucketID = bucket+1
            print ('Bucket', bucketID)

            #import everything needed to sort and predict
            pipeline_path = os.path.join(PATH, "%s/%s_%s/pipelines/pipeline_bucket_%s.joblib" % (dataset_ref, cls_method, method_name, bucketID))
            feat_comb_path = os.path.join(PATH, "%s/%s_%s/bucketers_and_encoders/feature_combiner_bucket_%s.joblib" % (dataset_ref, cls_method, method_name, bucketID))
            bucketer_path = os.path.join(PATH, "%s/%s_%s/bucketers_and_encoders/bucketer_bucket_%s.joblib" % (dataset_ref, cls_method, method_name, bucketID))
            cls_path = os.path.join(PATH, "%s/%s_%s/models/cls_bucket_%s.joblib" % (dataset_ref, cls_method, method_name, bucketID))

            predictor = joblib.load(pipeline_path)
            cls = joblib.load(cls_path)
            feature_combiner = joblib.load(feat_comb_path)
            bucketer = joblib.load(bucketer_path)

            #import data for bucket
            X_train_path = os.path.join(PATH, "%s/%s_%s/train_data/bucket_%s_prefixes.pickle" % (dataset_ref, cls_method, method_name, bucketID))
            Y_train_path = os.path.join(PATH, "%s/%s_%s/train_data/bucket_%s_labels.pickle" % (dataset_ref, cls_method, method_name, bucketID))
            X_test_path = os.path.join(PATH, "%s/%s_%s/test_data/bucket_%s_prefixes.pickle" % (dataset_ref, cls_method, method_name, bucketID))
            Y_test_path = os.path.join(PATH, "%s/%s_%s/test_data/bucket_%s_labels.pickle" % (dataset_ref, cls_method, method_name, bucketID))

            with open (X_train_path, 'rb') as f:
                dt_train_bucket = pickle.load(f)
            with open (Y_train_path, 'rb') as f:
                train_y = pickle.load(f)
            with open (X_test_path, 'rb') as f:
                dt_test_bucket = pickle.load(f)
            with open (Y_test_path, 'rb') as f:
                test_y = pickle.load(f)

            #import previously defined samples
            tn_path = os.path.join(PATH, "%s/%s_%s/samples/true_neg_bucket_%s_.pickle" % (dataset_ref, cls_method, method_name, bucketID))
            tp_path = os.path.join(PATH, "%s/%s_%s/samples/true_pos_bucket_%s_.pickle" % (dataset_ref, cls_method, method_name, bucketID))
            fn_path = os.path.join(PATH, "%s/%s_%s/samples/false_neg_bucket_%s_.pickle" % (dataset_ref, cls_method, method_name, bucketID))
            fp_path = os.path.join(PATH, "%s/%s_%s/samples/false_pos_bucket_%s_.pickle" % (dataset_ref, cls_method, method_name, bucketID))

            sample_instances = []

            with open (tn_path, 'rb') as f:
                tn_list = pickle.load(f)
            with open (tp_path, 'rb') as f:
                tp_list = pickle.load(f)
            #with open (fn_path, 'rb') as f:
            #    fn_list = pickle.load(f)
            #with open (fp_path, 'rb') as f:
            #    fp_list = pickle.load(f)

            #save results to a list
            sample_instances.append(tn_list)
            sample_instances.append(tp_list)
            #sample_instances.append(fn_list)
            #sample_instances.append(fp_list)

            print(cls)

Bucket 1
XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.5353160434296517, gamma=0,
              gpu_id=-1, importance_type='gain', interaction_constraints=None,
              learning_rate=0.05959997729599409, max_delta_step=0, max_depth=22,
              min_child_weight=3, missing=nan, monotone_constraints=None,
              n_estimators=500, n_jobs=0, num_parallel_tree=1, random_state=22,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=22,
              subsample=0.8342285287391545, tree_method=None,
              validate_parameters=False, verbosity=None)


In [30]:
input_

,AMOUNT_REQ,Case ID,label,Activity,Resource,lifecycle:transition,timesincemidnight,timesincelastevent,timesincecasestart,event_nr,month,weekday,hour,open_cases,Complete Timestamp,case_length,prefix_nr,orig_case_id
152221,65000.0,204712_30,regular,A_SUBMITTED-COMPLETE,112.0,COMPLETE,719.0,0.000000,0.000000,1.0,1.0,6.0,11.0,840.0,2012-01-29 11:59:31.378,40,30,204712
152222,65000.0,204712_30,regular,A_PARTLYSUBMITTED-COMPLETE,112.0,COMPLETE,719.0,0.010450,0.010450,2.0,1.0,6.0,11.0,840.0,2012-01-29 11:59:32.005,40,30,204712
152223,65000.0,204712_30,regular,W_Afhandelen leads-SCHEDULE,112.0,SCHEDULE,719.0,0.062167,0.072617,3.0,1.0,6.0,11.0,840.0,2012-01-29 11:59:35.735,40,30,204712
152224,65000.0,204712_30,regular,W_Afhandelen leads-START,112.0,START,669.0,1390.024383,1390.097000,4.0,1.0,0.0,11.0,874.0,2012-01-30 11:09:37.198,40,30,204712
152225,65000.0,204712_30,regular,A_PREACCEPTED-COMPLETE,11202.0,COMPLETE,674.0,5.325433,1395.422433,5.0,1.0,0.0,11.0,875.0,2012-01-30 11:14:56.724,40,30,204712
152226,65000.0,204712_30,regular,W_Completeren aanvraag-SCHEDULE,11202.0,SCHEDULE,674.0,0.000733,1395.423167,6.0,1.0,0.0,11.0,875.0,2012-01-30 11:14:56.768,40,30,204712
152227,65000.0,204712_30,regular,W_Afhandelen leads-COMPLETE,11202.0,COMPLETE,674.0,0.018633,1395.441800,7.0,1.0,0.0,11.0,875.0,2012-01-30 11:14:57.886,40,30,204712
152228,65000.0,204712_30,regular,W_Completeren aanvraag-START,11009.0,START,1026.0,351.825867,1747.267667,8.0,1.0,0.0,17.0,851.0,2012-01-30 17:06:47.438,40,30,204712
152229,65000.0,204712_30,regular,A_ACCEPTED-COMPLETE,11009.0,COMPLETE,1039.0,12.869117,1760.136783,9.0,1.0,0.0,17.0,852.0,2012-01-30 17:19:39.585,40,30,204712
152230,65000.0,204712_30,regular,O_SELECTED-COMPLETE,11009.0,COMPLETE,1048.0,8.924133,1769.060917,10.0,1.0,0.0,17.0,852.0,2012-01-30 17:28:35.033,40,30,204712


In [91]:
input_ = sample_instances[0][0]['input']
new_input = input_
row = new_input.index[new_input['Activity'] == 'A_ACCEPTED-COMPLETE']
new_input = new_input.replace('A_ACCEPTED-COMPLETE', np.nan)
#new_input = new_input.loc[row]['Activity']='A_CANCELLED-COMPLETE'
#row = input_[input_['Activity']=='A_ACCEPTED-COMPLETE']
#row['Activity']='A_CANCELLED-COMPLETE'
#row
#new_input = input_.append(row)
new_input

,AMOUNT_REQ,Case ID,label,Activity,Resource,lifecycle:transition,timesincemidnight,timesincelastevent,timesincecasestart,event_nr,month,weekday,hour,open_cases,Complete Timestamp,case_length,prefix_nr,orig_case_id
152221,65000.0,204712_30,regular,A_SUBMITTED-COMPLETE,112.0,COMPLETE,719.0,0.000000,0.000000,1.0,1.0,6.0,11.0,840.0,2012-01-29 11:59:31.378,40,30,204712
152222,65000.0,204712_30,regular,A_PARTLYSUBMITTED-COMPLETE,112.0,COMPLETE,719.0,0.010450,0.010450,2.0,1.0,6.0,11.0,840.0,2012-01-29 11:59:32.005,40,30,204712
152223,65000.0,204712_30,regular,W_Afhandelen leads-SCHEDULE,112.0,SCHEDULE,719.0,0.062167,0.072617,3.0,1.0,6.0,11.0,840.0,2012-01-29 11:59:35.735,40,30,204712
152224,65000.0,204712_30,regular,W_Afhandelen leads-START,112.0,START,669.0,1390.024383,1390.097000,4.0,1.0,0.0,11.0,874.0,2012-01-30 11:09:37.198,40,30,204712
152225,65000.0,204712_30,regular,A_PREACCEPTED-COMPLETE,11202.0,COMPLETE,674.0,5.325433,1395.422433,5.0,1.0,0.0,11.0,875.0,2012-01-30 11:14:56.724,40,30,204712
152226,65000.0,204712_30,regular,W_Completeren aanvraag-SCHEDULE,11202.0,SCHEDULE,674.0,0.000733,1395.423167,6.0,1.0,0.0,11.0,875.0,2012-01-30 11:14:56.768,40,30,204712
152227,65000.0,204712_30,regular,W_Afhandelen leads-COMPLETE,11202.0,COMPLETE,674.0,0.018633,1395.441800,7.0,1.0,0.0,11.0,875.0,2012-01-30 11:14:57.886,40,30,204712
152228,65000.0,204712_30,regular,W_Completeren aanvraag-START,11009.0,START,1026.0,351.825867,1747.267667,8.0,1.0,0.0,17.0,851.0,2012-01-30 17:06:47.438,40,30,204712
152229,65000.0,204712_30,regular,NaN,11009.0,COMPLETE,1039.0,12.869117,1760.136783,9.0,1.0,0.0,17.0,852.0,2012-01-30 17:19:39.585,40,30,204712
152230,65000.0,204712_30,regular,O_SELECTED-COMPLETE,11009.0,COMPLETE,1048.0,8.924133,1769.060917,10.0,1.0,0.0,17.0,852.0,2012-01-30 17:28:35.033,40,30,204712


In [65]:
row

Int64Index([152247, 152248, 152249, 152250], dtype='int64')

In [92]:
test_x_group= feature_combiner.fit_transform(input_) 
#test_x=np.transpose(test_x_group[0])
p1 = cls.predict_proba(test_x_group)[0][0]
p1

0.5014116

In [93]:
test_x_group= feature_combiner.fit_transform(new_input) 
#test_x=np.transpose(test_x_group[0])
p2 = cls.predict_proba(test_x_group)[0][0]
p2

0.5353737

In [94]:
lime_explanations = []
trainingdata = feature_combiner.fit_transform(dt_train_bucket);
class_names=['regular','deviant']# regular is 0, deviant is 1, 0 is left, 1 is right
lime_explainer = lime.lime_tabular.LimeTabularExplainer(trainingdata, feature_names = feature_combiner.get_feature_names(),
                                          class_names=class_names, discretize_continuous=True)
test_x_group= feature_combiner.fit_transform(input_) 
test_x=np.transpose(test_x_group[0])
for i in range(1):
    lime_exp = generate_lime_explanations(lime_explainer, test_x, cls, input_, max_feat = 10)
    lime_explanations.append(lime_exp.as_list())

In [78]:
difference = p1-p2
difference

-0.49342203

In [83]:
len(exp[0])

134

In [84]:
len(lime_exp.as_list())

134

In [63]:
tree_explainer = shap.TreeExplainer(cls)
test_x_group= feature_combiner.fit_transform(input_) 
test_x=np.transpose(test_x_group[0])
feat_list = feature_combiner.get_feature_names()
exp, rel_exp = create_samples(tree_explainer, exp_iter, test_x.reshape(-1, len(test_x)), feat_list, top = max_feat)


[[('agg__Resource_10932.0', 1.2384669, 1.2384669),
  ('static__AMOUNT_REQ', -0.9581396, 0.9581396),
  ('agg__timesincelastevent_max', -0.47295249, 0.47295249),
  ('agg__weekday_max', 0.4716128, 0.4716128),
  ('agg__Resource_11009.0', 0.3341523, 0.3341523),
  ('agg__timesincecasestart_max', -0.2978454, 0.2978454),
  ('agg__Activity_W_Afhandelen leads-SCHEDULE', -0.26826555, 0.26826555),
  ('agg__timesincemidnight_min', 0.23509617, 0.23509617),
  ('agg__timesincelastevent_sum', -0.21588735, 0.21588735),
  ('agg__open_cases_mean', -0.19332415, 0.19332415),
  ('agg__timesincemidnight_max', 0.14543395, 0.14543395),
  ('agg__Activity_O_SENT_BACK-COMPLETE', -0.13754527, 0.13754527),
  ('agg__open_cases_min', 0.13352077, 0.13352077),
  ('agg__Activity_O_SELECTED-COMPLETE', 0.10877956, 0.10877956),
  ('agg__hour_max', 0.103454255, 0.103454255),
  ('agg__Activity_W_Afhandelen leads-START', -0.08462943, 0.08462943),
  ('agg__Resource_11202.0', 0.08037823, 0.08037823),
  ('agg__open_cases_max', 0.06870748, 0.06870748),
  ('agg__Activity_W_Valideren aanvraag-SCHEDULE', -0.067746006, 0.067746006),
  ('agg__Activity_A_ACCEPTED-COMPLETE', 0.067030296, 0.067030296),
  ('agg__Activity_W_Completeren aanvraag-START', -0.06673386, 0.06673386),
  ('agg__Resource_112.0', -0.062282555, 0.062282555),
  ('agg__Activity_W_Nabellen offertes-COMPLETE', -0.060926404, 0.060926404),
  ('agg__timesincemidnight_mean', -0.060691036, 0.060691036),
  ('agg__weekday_std', 0.060583446, 0.060583446),
  ('agg__hour_std', 0.058540076, 0.058540076),
  ('agg__weekday_mean', 0.053386148, 0.053386148),
  ('agg__Activity_O_SENT-COMPLETE', -0.04597641, 0.04597641),
  ('agg__timesincemidnight_std', -0.044041418, 0.044041418),
  ('agg__hour_min', 0.038923025, 0.038923025),
  ('agg__weekday_min', -0.037064403, 0.037064403),
  ('agg__Activity_A_CANCELLED-COMPLETE', 0.035745565, 0.035745565),
  ('agg__timesincecasestart_sum', -0.035741087, 0.035741087),
  ('agg__hour_mean', 0.03419312, 0.03419312),
  ('agg__open_cases_std', 0.031646106, 0.031646106),
  ('agg__Resource_10929.0', -0.031438585, 0.031438585),
  ('agg__month_max', -0.02991438, 0.02991438),
  ('agg__Resource_11049.0', -0.02965099, 0.02965099),
  ('agg__Resource_10913.0', 0.029396666, 0.029396666),
  ('agg__month_mean', -0.02926916, 0.02926916),
  ('agg__Resource_11179.0', 0.02436194, 0.02436194),
  ('agg__Resource_11181.0', -0.023935227, 0.023935227),
  ('agg__weekday_sum', -0.022729877, 0.022729877),
  ('agg__Activity_W_Valideren aanvraag-START', -0.02128056, 0.02128056),
  ('agg__Activity_O_CANCELLED-COMPLETE', 0.018801305, 0.018801305),
  ('agg__Resource_11121.0', 0.018006798, 0.018006798),
  ('agg__hour_sum', -0.017136479, 0.017136479),
  ('agg__Resource_11180.0', 0.01613837, 0.01613837),
  ('agg__Activity_A_APPROVED-COMPLETE', -0.01588245, 0.01588245),
  ('agg__Resource_10861.0', 0.015406359, 0.015406359),
  ('agg__Activity_O_CREATED-COMPLETE', 0.01463643, 0.01463643),
  ('agg__Activity_W_Nabellen offertes-START', -0.01409816, 0.01409816),
  ('agg__Activity_W_Afhandelen leads-COMPLETE', -0.013927455, 0.013927455),
  ('agg__Resource_11119.0', 0.013341073, 0.013341073),
  ('agg__Activity_A_DECLINED-COMPLETE', 0.012804513, 0.012804513),
  ('agg__Resource_10939.0', 0.012554629, 0.012554629),
  ('agg__open_cases_sum', 0.01250986, 0.01250986),
  ('agg__Resource_10880.0', 0.01225498, 0.01225498),
  ('agg__timesincecasestart_std', -0.012076063, 0.012076063),
  ('agg__Resource_10899.0', -0.011865904, 0.011865904),
  ('agg__Activity_A_ACTIVATED-COMPLETE', -0.0110454215, 0.0110454215),
  ('agg__Resource_11203.0', -0.010295146, 0.010295146),
  ('agg__Resource_11169.0', -0.009860676, 0.009860676),
  ('agg__Resource_11000.0', -0.009789233, 0.009789233),
  ('agg__Resource_10881.0', -0.009603717, 0.009603717),
  ('agg__event_nr_mean', -0.009248129, 0.009248129),
  ('agg__Resource_11200.0', 0.009205993, 0.009205993),
  ('agg__timesincemidnight_sum', -0.009089798, 0.009089798),
  ('agg__Resource_10863.0', 0.009051847, 0.009051847)

In [102]:
features = []
for each in rel_exp:
    for feature in each:
        features.append(feature[0])
        
len(features)

import collections
counter = collections.Counter(features)
counter

Counter({'agg__Resource_10932.0': 10,
         'static__AMOUNT_REQ': 10,
         'agg__timesincelastevent_max': 10,
         'agg__weekday_max': 10,
         'agg__Resource_11009.0': 10,
         'agg__timesincecasestart_max': 10,
         'agg__Activity_W_Afhandelen leads-SCHEDULE': 10,
         'agg__timesincemidnight_min': 10,
         'agg__timesincelastevent_sum': 10,
         'agg__open_cases_mean': 10})

In [103]:
l_features = []
for each in lime_explanations:
    for feature in each:
        l_features.append(feature[0])
        
len(l_features)

import collections
counter = collections.Counter(l_features)
counter

Counter({'agg__Activity_A_CANCELLED-COMPLETE <= 0.00': 1,
         'agg__Activity_A_DECLINED-COMPLETE <= 0.00': 1,
         'agg__Activity_A_APPROVED-COMPLETE <= 0.00': 1,
         'agg__Activity_A_ACTIVATED-COMPLETE <= 0.00': 1,
         'agg__Activity_O_SENT_BACK-COMPLETE <= 0.00': 1,
         'agg__Resource_10138.0 <= 0.00': 1,
         'agg__Resource_10972.0 <= 0.00': 1,
         'agg__Resource_10779.0 <= 0.00': 1,
         'agg__Activity_W_Wijzigen contractgegevens-SCHEDULE <= 0.00': 1,
         'agg__Resource_other <= 0.00': 1})

In [104]:
lime_explanations

[[('agg__Activity_A_CANCELLED-COMPLETE <= 0.00', 0.5009202902423527),
  ('agg__Activity_A_DECLINED-COMPLETE <= 0.00', 0.39581680589476054),
  ('agg__Activity_A_APPROVED-COMPLETE <= 0.00', -0.30751626085121175),
  ('agg__Activity_A_ACTIVATED-COMPLETE <= 0.00', -0.28695171495247557),
  ('agg__Activity_O_SENT_BACK-COMPLETE <= 0.00', -0.2549704413009707),
  ('agg__Resource_10138.0 <= 0.00', -0.22718675191020934),
  ('agg__Resource_10972.0 <= 0.00', -0.21851929727936004),
  ('agg__Resource_10779.0 <= 0.00', -0.1853387342018481),
  ('agg__Activity_W_Wijzigen contractgegevens-SCHEDULE <= 0.00',
   0.13532290887968482),
  ('agg__Resource_other <= 0.00', -0.10581558354724857)]]